In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df= pd.read_csv('https://github.com/armaf002/Food-price/raw/main/preprocessed-data.csv')

In [3]:
df['date']=pd.to_datetime(df['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64221 entries, 0 to 64220
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              64221 non-null  datetime64[ns]
 1   State             64221 non-null  object        
 2   Local_Government  64221 non-null  object        
 3   Market_Name       64221 non-null  object        
 4   latitude          64221 non-null  float64       
 5   longitude         64221 non-null  float64       
 6   category          64221 non-null  object        
 7   commodity         64221 non-null  object        
 8   unit              64221 non-null  object        
 9   priceflag         64221 non-null  object        
 10  pricetype         64221 non-null  object        
 11  currency          64221 non-null  object        
 12  price             64221 non-null  float64       
 13  usdprice          64221 non-null  float64       
 14  year              6422

In [4]:
categorical_features= [x for x in df.columns if df[x].dtype=='O']

for col in categorical_features:
    print(col)
    print(df[col].unique())
    print("\n")

State
['Katsina' 'Sokoto' 'Borno' 'Kano' 'Jigawa' 'Oyo' 'Lagos' 'Kaduna'
 'Zamfara' 'Abia' 'Gombe' 'Kebbi' 'Adamawa' 'Yobe']


Local_Government
['Jibia' 'Gada' "Mai'Adua" 'Mobbar' 'Dawakin Tofa' 'Maigatari'
 'Ibadan North' 'Maiduguri' 'Kosofe' 'Giwa' 'Kaura Namoda' 'Oboma Ngwa'
 'Akko' 'Kaugama' 'Lere' 'Dandume' 'Gwandu' 'Hong' 'Biu' 'Damaturu'
 'Potiskum' 'Konduga' 'Geidam' 'Gujba' 'Jakusko' 'Karasuwa' 'Nguru'
 'Yunusari' 'Yusufari' 'Borsari' 'Gulani']


Market_Name
['Jibia (CBM)' 'Illela (CBM)' 'Mai Adoua (CBM)' 'Damassack (CBM)'
 'Dawanau' 'Mai Gatari (CBM)' 'Ibadan' 'Maiduguri' 'Lagos' 'Giwa'
 'Kaura Namoda' 'Aba' 'Gombe' 'Gujungu' 'Saminaka' 'Dandume' 'Gwandu'
 'Mubi' 'Biu' 'Damaturu' 'Potiskum' 'Abba Gamaram' 'Baga Road'
 'Bullunkutu' 'Budum' 'Custom' 'Kusawam Shanu' 'Monday' 'Tashan Bama'
 'Bolori Stores' 'Damaturu (Sunday Market)' 'Geidam' 'Gujba (Buni Yadi)'
 'Jakusko' 'Bade (Gashua)' 'Nguru' 'Yunusari' 'Yusufari' 'Bursari'
 'Gulani (Tettaba)']


category
['cereals and tubers'

# Treating Outlier by price of each Commodity

In [5]:
# Average price based on commodity and unit of food items
df.groupby(['commodity', 'unit'])['price'].mean()

commodity                    unit      
Bananas                      1KG              16.263438
Beans (niebe)                1KG             252.294500
Beans (red)                  1KG              38.065116
Beans (white)                1KG              34.919735
Bread                        Unit            264.564173
Cassava meal (gari, yellow)  1KG             229.160238
Cowpeas                      1KG              27.399075
Cowpeas (brown)              1KG             313.359220
Cowpeas (white)              1KG             284.857645
Eggs                         30 pcs         1217.207886
Fish                         1KG              30.154625
Fuel (diesel)                L               281.256098
Fuel (petrol-gasoline)       L               153.291524
Gari (white)                 1KG             195.490945
Groundnuts                   1KG              41.877470
Groundnuts (shelled)         1KG             368.037265
Maize                        1KG             172.553387
Maize (w

In [6]:
# Grouping the dataframe into different dataframes based on commodity and unit

lst_var_df=[]

for variables in df['commodity'].unique():
    if len(df[df['commodity']== variables]['unit'].unique()) ==1:
        var_df= df[df['commodity']== variables]
        lst_var_df.append(var_df)
    else:
        var_df= df[df['commodity']== variables]
        for units in var_df['unit'].unique():
            var_dfs=var_df[var_df['unit']==units]
            lst_var_df.append(var_dfs)


In [7]:
# outlier removal from each dataframe

no_outlier_df= []

for data in lst_var_df:
    df_sub= pd.DataFrame(data['price'])
    Q1= df_sub.quantile(0.25)
    Q3= df_sub.quantile(0.75)
    IQR = Q3 - Q1
    cleaned_df = df_sub[~((df_sub < (Q1 - 1.5 * IQR)) |(df_sub > (Q3 + 1.5 * IQR))).any(axis=1)]
    no_outlier_df.append(cleaned_df)

In [8]:
#collecting all the index of leftover rows after outliers are removed

no_outlier_indices=[]
for x in range(len(no_outlier_df)):
    index_list=no_outlier_df[x].index.values.tolist()
    no_outlier_indices.append(index_list)


In [9]:
# proper list of all index needed

indices=[]
for i in range(len(no_outlier_indices)):
    indices += no_outlier_indices[i]

len(indices)


61869

In [10]:
len(df)

64221

After removing all ouliers, 61869 rows were remaining, out of 64221 rows

In [11]:
# new dataframe
no_outlier_df= df.iloc[indices]

# **Feature Transformation**

In [12]:
# creating weekday column from date column
no_outlier_df['day']=no_outlier_df['date'].dt.strftime('%A')
no_outlier_df.head()

<ipython-input-12-08ce57857e98>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_outlier_df['day']=no_outlier_df['date'].dt.strftime('%A')


,date,State,Local_Government,Market_Name,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice,year,month,season,Inflation,day
0,2002-01-15,Katsina,Jibia,Jibia (CBM),13.080,7.240,cereals and tubers,Maize,1KG,actual,Wholesale,NGN,175.92,1.5525,2002,January,Dry Season,12.876579,Tuesday
9,2002-02-15,Katsina,Jibia,Jibia (CBM),13.080,7.240,cereals and tubers,Maize,1KG,actual,Wholesale,NGN,169.76,1.4826,2002,February,Dry Season,12.876579,Friday
14,2002-03-15,Katsina,Jibia,Jibia (CBM),13.080,7.240,cereals and tubers,Maize,1KG,actual,Wholesale,NGN,181.94,1.5767,2002,March,Early Rainy Season,12.876579,Friday
19,2002-03-15,Sokoto,Gada,Illela (CBM),13.645,5.278,cereals and tubers,Maize,1KG,actual,Wholesale,NGN,171.25,1.4840,2002,March,Early Rainy Season,12.876579,Friday
23,2002-04-15,Katsina,Jibia,Jibia (CBM),13.080,7.240,cereals and tubers,Maize,1KG,actual,Wholesale,NGN,190.22,1.6441,2002,April,Early Rainy Season,12.876579,Monday


## **Droping unuseful columns**

---


*   currency - doesn't affect price its a constant

* "local government", "market" and "state" columns since we have longitude and latitude columns
*    date - dropped because Theres year and month and day column available

In [13]:
df_new=no_outlier_df.drop(['date', 'State', 'Local_Government', 'Market_Name','currency'], axis=1)

In [14]:
categorical_feature= [col for col in df_new.columns if df_new[col].dtype=='O']
categorical_feature

['category',
 'commodity',
 'unit',
 'priceflag',
 'pricetype',
 'month',
 'season',
 'day']

In [15]:
for col in categorical_feature:
    print(f"number of uniques variables in {col} column is {len(df_new[col].unique())}")

number of uniques variables in category column is 8
number of uniques variables in commodity column is 42
number of uniques variables in unit column is 5
number of uniques variables in priceflag column is 3
number of uniques variables in pricetype column is 2
number of uniques variables in month column is 12
number of uniques variables in season column is 4
number of uniques variables in day column is 7


## **Handling categorical features**

---
* handling all columns with less than 5 unique features using one hot encoding
* Mapping month and day column to follow sequential order (ordinal variables)


In [16]:
# to get the columns with 5 and fewer unique variables
dummy_col=[]
for col in categorical_feature:
    var_len= len(df_new[col].unique())
    if var_len<=5:
        dummy_col.append(col)

# applying one hot encoding to columns with 5 and fewer uniques variables
df_new= pd.get_dummies(df_new, columns = dummy_col, drop_first=True)
df_new.head()

,latitude,longitude,category,commodity,price,usdprice,year,month,Inflation,day,unit_1KG,unit_30 pcs,unit_L,unit_Unit,"priceflag_actual,aggregate",priceflag_aggregate,pricetype_Wholesale,season_Early Rainy Season,season_Late Rainy Season,season_Peak Rainy Season
0,13.080,7.240,cereals and tubers,Maize,175.92,1.5525,2002,January,12.876579,Tuesday,1,0,0,0,0,0,1,0,0,0
9,13.080,7.240,cereals and tubers,Maize,169.76,1.4826,2002,February,12.876579,Friday,1,0,0,0,0,0,1,0,0,0
14,13.080,7.240,cereals and tubers,Maize,181.94,1.5767,2002,March,12.876579,Friday,1,0,0,0,0,0,1,1,0,0
19,13.645,5.278,cereals and tubers,Maize,171.25,1.4840,2002,March,12.876579,Friday,1,0,0,0,0,0,1,1,0,0
23,13.080,7.240,cereals and tubers,Maize,190.22,1.6441,2002,April,12.876579,Monday,1,0,0,0,0,0,1,1,0,0


In [17]:
df_new['month']=df_new['month'].map({'January':1, 'February':2, 'March':3, 'April':4, 'May':5, 'June':6, 'July':7,
       'August':8, 'September':9, 'October':10, 'November':11, 'December':12})
df_new['day']=df_new['day'].map({'Sunday':1 ,'Monday': 2,'Tuesday':3, 'Wednesday':4,'Thursday': 5, 'Friday':6,'Saturday':7})

In [18]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

cat_feature= [col for col in df_new.columns if df_new[col].dtype=='O']
for feature in cat_feature:
  df_new[feature]=le.fit_transform(df_new[feature])

In [19]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61869 entries, 0 to 48926
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   latitude                    61869 non-null  float64
 1   longitude                   61869 non-null  float64
 2   category                    61869 non-null  int64  
 3   commodity                   61869 non-null  int64  
 4   price                       61869 non-null  float64
 5   usdprice                    61869 non-null  float64
 6   year                        61869 non-null  int64  
 7   month                       61869 non-null  int64  
 8   Inflation                   61869 non-null  float64
 9   day                         61869 non-null  int64  
 10  unit_1KG                    61869 non-null  uint8  
 11  unit_30 pcs                 61869 non-null  uint8  
 12  unit_L                      61869 non-null  uint8  
 13  unit_Unit                   618

No other object column avilable
Categorical features successfully encoded!